In [1]:
# load all packages in Julia
import Pkg; Pkg.add("VegaLite"); Pkg.add("PrettyTables")
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

    Updating registry at `C:\Users\44780\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`


In [51]:
#open csv files to define sets, lines, gens and and loads/demand
datadir = joinpath("ReducedNetwork") 
gens = CSV.read(joinpath(datadir,"UpdateGenRedNet.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"UpdateTransRedNet.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"demandRedNet.csv"), DataFrame);
a = CSV.read(joinpath(datadir,"A_matrix.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids #changed to read columns rather than rows
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters


#lines.capacity = 1lines.capacity
# calculate simple susceptance, ignoring resistance as earlier 
#lines.b = 1 ./ lines.reactance

# keep only a single time period
#loads = loads[:,["connnode","interval-1_load"]]
#rename!(loads,"interval-1_load" => "demand");

lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b,column9,column10,column11,column12,column13
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Missing,Missing,Missing,Missing,Int64
1,1,2,0.0122,0.02,1,525,1,0.0856,missing,missing,missing,missing,525
2,1,3,0.007,0.15,1,132,2,0.052,missing,missing,missing,missing,132
3,1,2,0.0122,0.02,1,525,3,0.2844,missing,missing,missing,missing,525
4,1,3,0.007,0.15,1,132,4,0.052,missing,missing,missing,missing,132
5,2,4,0.0004,0.065,1,760,5,0.4454,missing,missing,missing,missing,760
6,2,4,0.0004,0.065,1,760,6,0.5545,missing,missing,missing,missing,760
7,3,4,0.003,0.041,1,648,7,0.0044,missing,missing,missing,missing,648
8,3,4,0.003,0.041,1,648,8,0.044,missing,missing,missing,missing,648
9,4,5,0.00211,0.0135,1,1090,9,0.1174,missing,missing,missing,missing,1090


In [62]:
sum(gens.pgmax)

237270.089959114

In [16]:
a

Row,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,id
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
4,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
5,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
6,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
7,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
8,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
9,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9


In [53]:
sum(loads.demand)

-73847.1343515

In [42]:
loads[loads.connnode .== 2,:demand]

1-element Vector{Float64}:
 -624.6954815

In [44]:
loads[loads.connnode .== 1,:demand][1]
 

-390.3

In [5]:
loads

Row,connnode,demand,column3,column4,column5,column6,column7
,Int64,Float64,Missing,Missing,Missing,String31,Int64
1,1,-390.3,missing,missing,missing,Beauly,1
2,2,-624.695,missing,missing,missing,Peterhead,2
3,3,-202.172,missing,missing,missing,Errochty,3
4,4,-1828.53,missing,missing,missing,Denny/Bonnybridge,4
5,7,-2168.74,missing,missing,missing,Neilston,7
6,6,-660.162,missing,missing,missing,Strathaven,6
7,5,-572.218,missing,missing,missing,Torness,5
8,10,-162.219,missing,missing,missing,Eccles,10
9,9,-581.174,missing,missing,missing,Harker,9


In [56]:
#attempt 2 using PyPSA-GB network - works properly for the 36 bus network
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode))) 
     #set of all lines
    L = lines.id
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system 
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[L]        # flows between all pairs of nodes
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gens[g,:c1] * GEN[g] for g in G)
    )
    
 # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:id]) 
            + loads[loads.connnode .== i,:demand][1] 
        == sum(a[l,i]*FLOW[l] for l in L) #ammended from tonode to :id
    )
 #loads[loads.connnode .== i,:demand]
    
    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])
    
    # Flow constraints on each branch 
    #@constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
     #       FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
      #      baseMVA * lines[l,:b] * 
       #     (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
    #)
    
    # Max line flow constraints
    #@constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
     #       FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
      #      lines[l,:capacity]
    #) 

    @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)],
           -lines[l, :capacity] <= FLOW[l] <= lines[l,:capacity]
    ) 

    @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
            FLOW[l] == lines[l, :b] * sum(THETA[n] * a[l, n] for n in N)
    )



    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.id] #ammended from connonde to id
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee (generic function with 1 method)

In [20]:
sum(loads.demand)

-56325.86

In [57]:
solution = dcopf_ieee(gens, lines, loads);

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
127 rows, 448 cols, 810 nonzeros
123 rows, 242 cols, 596 nonzeros
123 rows, 241 cols, 595 nonzeros
Presolve : Reductions: rows 123(-439); columns 241(-222); elements 595(-669)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -3.5731968984e-02 Pr: 123(455571); Du: 0(1.87959e-11) 0s
        148     6.4189430160e+04 Pr: 0(0); Du: 0(3.33067e-15) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 148
Objective value     :  6.4189430160e+04
HiGHS run time      :          0.01


In [58]:
solution.generation #pypsa-gb results below

Row,node,gen
,Int64,Float64
1,1,0.0
2,1,0.0
3,1,0.0
4,1,1252.66
5,1,0.0
6,1,0.0
7,1,0.0
8,1,0.0
9,1,0.0


In [59]:
gentest = solution.generation

Row,node,gen
,Int64,Float64
1,1,0.0
2,1,0.0
3,1,0.0
4,1,1252.66
5,1,0.0
6,1,0.0
7,1,0.0
8,1,0.0
9,1,0.0


In [51]:
using CSV

In [60]:
CSV.write(joinpath(datadir,"GenTestCap.csv"), gentest)

"ReducedNetwork\\GenTestCap.csv"

In [49]:
sum(gentest.gen)

35724.83275248658

In [37]:
solution.prices

Row,node,value
,Int64,Float64
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0
5,5,1.0
6,6,1.0
7,7,1.0
8,8,1.0
9,9,1.0


In [61]:
solution.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,15358.1
2,1,3,9925.87
3,1,2,51026.1
4,1,3,9925.87
5,2,4,1743.72
6,2,4,2170.84
7,3,4,-33.2231
8,3,4,-332.231
9,4,5,-57999.4
